In [46]:
import pickle
import numpy as np
import scipy
import scipy.interpolate
from dmp import DMP

In [48]:
dmp = DMP()

In [82]:
dmp.basis_centers.shape

(30,)

In [81]:
dmp.basis_variances.shape

(30,)

In [78]:
dmp.K

array([[10.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 10.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 10.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 10.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 10.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.]])

In [52]:
a = np.array([[1, 1, 1], [2, 2, 2]])
b = np.array([1, 2])

In [54]:
a.shape, b.shape

((2, 3), (2,))

In [84]:
b.reshape((-1, 1)).shape

(2, 1)

In [87]:
a[:, :, None].shape

(2, 3, 1)

In [74]:
a / b.reshape((len(b), -1))

array([[1., 1., 1.],
       [1., 1., 1.]])

In [76]:
tau = T[:, -1]
tau

array([0.56, 0.84, 0.72, 0.8 , 0.76, 0.68, 0.96, 0.68, 0.64, 0.76, 0.68,
       0.88, 0.92, 0.76, 0.76, 0.72, 1.04, 1.16, 0.88, 1.12, 0.6 , 0.72,
       0.52, 0.76, 0.72, 0.72, 0.6 , 0.72, 0.76, 0.72, 0.68, 0.68, 0.6 ,
       0.88, 1.  , 0.56, 0.72, 0.92, 0.8 , 0.72, 0.76, 0.72, 0.64, 0.84,
       0.68, 0.72, 0.68, 0.84, 0.64, 0.72, 0.56, 0.72, 0.6 , 0.68, 0.8 ,
       0.68, 0.84, 0.64, 0.84, 0.68, 0.64, 0.92, 0.8 , 0.56, 0.84, 0.84,
       0.72, 0.64, 0.72, 0.72, 0.6 , 0.72, 0.68, 0.76, 0.72, 0.8 , 0.88,
       0.88, 0.68, 0.72, 0.76, 0.6 , 0.84, 0.84, 0.6 , 0.76, 0.92])

In [33]:
def load_dataset(dataset_path="data/demos.pkl"):
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    return [traj[:, :6] for traj in dataset["trajectories"]] # 1st 6 elements are joint angle

In [88]:
data = load_dataset()

In [38]:
def interpolate(trajectories, initial_dt=0.04):
    """
    Combine the given variable length trajectories into a fixed length array
    by interpolating shorter arrays to the maximum given sequence length.

    trajectories: A list of N arrays of shape (T_i, num_dofs) where T_i is the number
        of time steps in trajectory i
    initial_dt: A scalar corresponding to the duration of each time step.

    Returns: A numpy array of shape (N, max_i T_i, num_dofs)
    """
    length = max(len(traj) for traj in trajectories)
    dofs = trajectories[0].shape[1]

    X = np.zeros((len(trajectories), length, dofs))
    T = np.zeros((len(trajectories), length))

    for ti, traj in enumerate(trajectories):
        t = np.arange(len(traj)) * initial_dt
        t_new = np.linspace(0, t.max(), length)
        T[ti, :] = t_new
        for deg in range(dofs):
            path_gen = scipy.interpolate.interp1d(t, traj[:, deg])
            X[ti, :, deg] = path_gen(t_new)
    return X, T

In [90]:
X, T = interpolate(data)

In [93]:
X[0].shape, T[0].shape

((30, 6), (30,))

In [43]:
X.shape

(87, 30, 6)

In [44]:
T.shape

(87, 30)

In [26]:
len(data), data[0].shape

(87, (15, 6))

In [31]:
for i in range(len(data)):
    print(data[i].shape)

(15, 6)
(22, 6)
(19, 6)
(21, 6)
(20, 6)
(18, 6)
(25, 6)
(18, 6)
(17, 6)
(20, 6)
(18, 6)
(23, 6)
(24, 6)
(20, 6)
(20, 6)
(19, 6)
(27, 6)
(30, 6)
(23, 6)
(29, 6)
(16, 6)
(19, 6)
(14, 6)
(20, 6)
(19, 6)
(19, 6)
(16, 6)
(19, 6)
(20, 6)
(19, 6)
(18, 6)
(18, 6)
(16, 6)
(23, 6)
(26, 6)
(15, 6)
(19, 6)
(24, 6)
(21, 6)
(19, 6)
(20, 6)
(19, 6)
(17, 6)
(22, 6)
(18, 6)
(19, 6)
(18, 6)
(22, 6)
(17, 6)
(19, 6)
(15, 6)
(19, 6)
(16, 6)
(18, 6)
(21, 6)
(18, 6)
(22, 6)
(17, 6)
(22, 6)
(18, 6)
(17, 6)
(24, 6)
(21, 6)
(15, 6)
(22, 6)
(22, 6)
(19, 6)
(17, 6)
(19, 6)
(19, 6)
(16, 6)
(19, 6)
(18, 6)
(20, 6)
(19, 6)
(21, 6)
(23, 6)
(23, 6)
(18, 6)
(19, 6)
(20, 6)
(16, 6)
(22, 6)
(22, 6)
(16, 6)
(20, 6)
(24, 6)


In [30]:
np.stack(data, 0).shape

ValueError: all input arrays must have the same shape

In [13]:
with open('data/demos.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [18]:
a = np.array([[1, 1, 1, 1, 1], [2, 2, 2, 2, 2]])
a

array([[1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2]])

In [19]:
a.shape

(2, 5)

In [20]:
a[:, -1]

array([1, 2])

In [21]:
a = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
b = np.array([3, 3, 3])

In [22]:
a / b

array([[0.33333333, 0.33333333, 0.33333333],
       [0.66666667, 0.66666667, 0.66666667],
       [1.        , 1.        , 1.        ]])